In [16]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
import math
import pickle

In [17]:
data = pd.read_csv('./../data/round.csv')
data = data.loc[data['Permanent_Tournament_#']!=470] ## this is the match play championship, no round scores available

In [18]:
data.shape

(257759, 12)

In [20]:
data.head(3)

,Tournament_Year,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Round_Score,End_of_Event_Pos._(text),Event_Name,Strokes_In_Tournament,Rounds_In_Tournament,Finishing_Position
0,2003,16,656,1320,"Estes, Bob",1,66,T6,Mercedes Championships,271,4,0.208333
1,2003,16,656,1361,"Forsman, Dan",1,69,T21,Mercedes Championships,279,4,0.611111
2,2003,16,656,1761,"Mattiace, Len",1,75,24,Mercedes Championships,280,4,0.666667


In [21]:
with open('./../PickleFiles/num_to_ind_round.pkl','r') as pickleFile:
    num_to_inds = pickle.load(pickleFile)

In [22]:
data.insert(5,'Player_Index',[num_to_inds[num] for num in data.Player_Number])

In [23]:
tups = data.drop_duplicates(['Tournament_Year','Permanent_Tournament_#'])[['Tournament_Year','Permanent_Tournament_#']].values.tolist()
tournament_groups = {tuple(tup):u/4 for u,tup in enumerate(tups)}

In [24]:
data.insert(len(data.columns),'Tournament_Group',[tournament_groups[tuple(tup)] 
                                                  for tup in data[['Tournament_Year','Permanent_Tournament_#']].values.tolist()])

In [25]:
n_tournament_groups = len(pd.unique(data.Tournament_Group))

In [26]:
n_tournament_groups

158

In [27]:
def save_sparse_csc(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

In [28]:
for (year,tourn_group),df in data.groupby(['Tournament_Year','Tournament_Group'],sort=False):
    A,G = np.zeros((n_players,n_players)),np.zeros((n_players,n_players))
    
    for (tourn,round),df_ in df.groupby(['Course_#','Round_Number']):
        A[np.ix_(df_.Player_Index.values,df_.Player_Index.values)] += \
                np.outer(1/df_.Round_Score.values.astype(float),df_.Round_Score.values)
        G[np.ix_(df_.Player_Index.values,df_.Player_Index.values)] += 1.
    
    np.fill_diagonal(A,0)
    np.fill_diagonal(G,0)
    A = csc_matrix(A)
    G = csc_matrix(G)
    save_sparse_csc('./../rounds/%dA' % (tourn_group),A)
    save_sparse_csc('./../rounds/%dG' % (tourn_group),G)